In [1]:
!pip install bs4

     |████████████████████████████████| 115 kB 7.0 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=3afe4574f5987f025347441326a83f46c2c52730a094ca1c04773beee055f827
  Stored in directory: /home/jupyter/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [2]:
import requests
import re
import json
from bs4 import BeautifulSoup

def lyrics_from_song_api_path(song_api_path, headers):
    base_url = "http://api.genius.com"
    song_url = base_url + song_api_path
    response = requests.get(song_url, headers=headers)
    json = response.json()
    path = json["response"]["song"]["path"]
    #html scraping
    page_url = "http://genius.com" + path
    page = requests.get(page_url)
    #removing script symbols
    page=re.sub('<br/>', '\n', page.text)
    #removing words [Verse...] and [Chorus...]
    page=re.sub('\[.*.\]', ' ', page)
    #removing extra spaces
    page=re.sub('\n{2}', ' ', page)
    html = BeautifulSoup(page, "html.parser")
    #finding tag called 'lyrics'!
    lyrics = html.find("div", class_=re.compile("Lyrics|^lyrics$|Lyrics_Root")).get_text() #updated css where the lyrics are based in HTML
    #removing Test Section stuck at the front
    lyrics=re.sub('test placeholderSaveSave.*','',lyrics)
    #removing Embedded Section stuck at the end
    lyrics=re.sub('[0-9].*.EmbedShare URLCopyEmbedCopy|EmbedShare URLCopyEmbedCopy', ' ', lyrics)
    #removing (') as it confuses code
    lyrics=re.sub('\'','', lyrics)
    #removing all line breaks (\n)
    lyrics=list(lyrics)
    lyrics = [", " if elem == "\n" else elem for elem in lyrics]
    lyrics.remove(", ")
    lyrics="".join(lyrics)
    return lyrics

def get_info_genius_API(song_title, artist_name, headers):
    base_url = "http://api.genius.com"
    search_url = base_url + "/search"
    song_title=re.sub('\(.*\)', '', song_title)
    data = {'q': song_title+', '+artist_name}
    response = requests.get(search_url, params=data, headers=headers)
    json = response.json()
    song_info = None
    for hit in json["response"]["hits"]:
        hit_title=hit["result"]["title"]
        
        primary_hit_title=re.sub('\(.*\)', '', hit_title).lower().strip()
        primary_song_title=re.sub('-.*','',song_title).lower().strip()
        
        hit_artist_name=hit["result"]["primary_artist"]["name"]
        primary_artist_name=re.sub('(,|&).*', '', artist_name).lower()
        primary_hit_artist_name=re.sub('(,|&).*', '', hit_artist_name).lower()  
        
        if (primary_hit_title== primary_song_title): return hit
        elif (primary_hit_artist_name == primary_artist_name): return hit
    return None 


def get_lyrics_genius_API(song_title, artist_name):
    client_access_token='Q56ffH3XEyFV-Obd4uUhh2E8aZvC3ncgNXRPef5qP1nhyJaLUbr5ENYuZNtB472_'
    headers = {'Authorization': 'Bearer '+client_access_token}
    song_info = get_info_genius_API(song_title, artist_name, headers)
    if song_info: 
        song_api_path = song_info["result"]["api_path"]
        return lyrics_from_song_api_path(song_api_path, headers)
    else:
        artist_name=re.sub('(,|&).*', '', artist_name)
        song_info = get_info_genius_API(song_title, artist_name, headers)
        if song_info: 
            song_api_path = song_info["result"]["api_path"]
            return lyrics_from_song_api_path(song_api_path, headers)
    print("Error: No Match "+song_title+" by "+artist_name)
    return None
    
def get_lyrics_dict(json_data):
    lyrics_json={}
    for index, value in json_data.items():
        track_lyrics_dict={}
        artist = value['artist']
        track =  value['song']
        song_id= value['song_id']
        lyrics=get_lyrics_genius_API(song_title= track, artist_name= artist)
        try:
            track_lyrics_dict["song_id"]=song_id
            track_lyrics_dict["lyrics"]=lyrics
            lyrics_json[index]=track_lyrics_dict
        except TypeError:
            print(index)
            continue
    return lyrics_json
            
def read_json_file(file_name):
    file = open(file_name,"r")
    s = file.read()
    json_data = json.loads(s)
    file.close()
    return json_data

def spotify_json_to_lyrics_json(file_name):
    spotify_outputs_path='../w205-project3/outputs/spotify/'
    json_data=read_json_file(spotify_outputs_path+file_name)
    lyrics_dict=get_lyrics_dict(json_data)
    with open('../w205-project3/outputs/genius/genuis_lyrics_'+file_name, "w") as outfile: 
        json.dump(lyrics_dict, outfile)

In [4]:
spotify_json_to_lyrics_json('top_50_US.json')

In [4]:
spotify_json_to_lyrics_json('top_50_JP.json')

Error: No Match 僕のスピな人 by Maharajan
Error: No Match Given-Taken - Japanese Ver. by ENHYPEN
Error: No Match U+ by にしな


In [5]:
spotify_json_to_lyrics_json('top_50_VN.json')

Error: No Match double take by dhruv
Error: No Match The Playah (feat. SlimV) - Special Performance by Soobin Hoang Son
Error: No Match 3 1 0 7 - 2 (feat. W/N & Nau) by W/N
Error: No Match Tháng Mấy Em Nhớ Anh? by Ha Anh Tuan
Error: No Match Ke Cap Gap Ba Gia (feat. BINZ) by Hoang Thuy Linh
Error: No Match từ chối nhẹ nhàng thôi by Bich Phuong


In [6]:
spotify_json_to_lyrics_json('top_50_Ger.json')

Error: No Match It Ain't Me (with Selena Gomez) by Kygo
Error: No Match Murcielago by KC Rebell
Error: No Match You Don't Know Me - Radio Edit by Jax Jones
Error: No Match Two Princes - Radio Mix by Fake Pictures


**Manually searching Genius API for songs that were not found**

Checking unfound JP top 50 songs 

In [7]:
genius_json_path='../w205-project3/outputs/genius/genuis_lyrics_top_50_'
JP_dict=read_json_file(genius_json_path+'JP.json')

In [8]:
JP_dict['32']['lyrics']=get_lyrics_genius_API(song_title="Maharajan - Boku no supinahito 僕のスピな人"  \
                                            , artist_name="Genius English Translations")
JP_dict['42']['lyrics']=get_lyrics_genius_API(song_title="Given-Taken (Japanese Ver.)", \
                                              artist_name="ENHYPEN (엔하이픈)")
#Song index 46, U+ by にしな is not in Genius

In [9]:
with open(genius_json_path+'JP.json', "w") as outfile: 
    json.dump(JP_dict, outfile)

Checking unfound VN top 50 songs 

In [10]:
genius_json_path='../w205-project3/outputs/genius/genuis_lyrics_top_50_'
VN_dict=read_json_file(genius_json_path+'VN.json')

In [11]:
client_access_token='Q56ffH3XEyFV-Obd4uUhh2E8aZvC3ncgNXRPef5qP1nhyJaLUbr5ENYuZNtB472_'
headers = {'Authorization': 'Bearer '+client_access_token}

#double take by dhruv, lyrics scraped directly from song_api_path
VN_dict['3']['lyrics']=lyrics_from_song_api_path(song_api_path="/songs/4651202", \
                                             headers=headers)

#Song index 9, The Playah by Soobin Hoang Son is not in Genius

VN_dict['26']['lyrics']=get_lyrics_genius_API(song_title= "3 1 0 7 -2"
                                            , artist_name= "Duongg")
VN_dict['39']['lyrics']=get_lyrics_genius_API(song_title= "Tháng Mấy Em Nhớ Anh"\
                                            , artist_name= "Hà Anh Tuấn")
VN_dict['43']['lyrics']=get_lyrics_genius_API(song_title= "Kẻ Cắp Gặp Bà Già (Diamond Cut Diamond)"\
                                            , artist_name= "Hoàng Thuỳ Linh")
VN_dict['46']['lyrics']=get_lyrics_genius_API(song_title= "từ chối nhẹ nhàng thôi"\
                                            , artist_name= "Bích Phương & Phúc Du")


In [12]:
with open(genius_json_path+'VN.json', "w") as outfile: 
    json.dump(VN_dict, outfile)